# Overlap Graphs

https://rosalind.info/problems/grph/

In [1]:
import sys
print(sys.version)

3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]


## Solutions

### Simple solution O(N^2)

In [18]:
def parse_fasta(fasta_str: str) -> dict:
    entries = fasta_str.split('>')
    fasta_dict = {}
    for entry in entries[1:]:
        lines = entry.strip().split('\n')
        header = lines[0]
        sequence = ''.join(lines[1:])
        fasta_dict[header] = sequence
    return fasta_dict


def overlap_graph(data: str, k: int = 3) -> list[tuple[str, str]]:
    """
    Find the overlap graph for the given DNA strings in FASTA format
    """
    parsed_data = parse_fasta(data)
    adjacency_list = []
    for i, (label1, seq1) in enumerate(parsed_data.items()):
        for j, (label2, seq2) in enumerate(parsed_data.items()):
            if i != j and seq1[-k:] == seq2[:k]:
                adjacency_list.append((label1, label2))
    return adjacency_list

fasta = """
>Rosalind_0498
AAATAAA
>Rosalind_2391
AAATTTT
>Rosalind_2323
TTTTCCC
>Rosalind_0442
AAATCCC
>Rosalind_5013
GGGTGGG
"""

for ans in overlap_graph(fasta):
    print(*ans)

Rosalind_0498 Rosalind_2391
Rosalind_0498 Rosalind_0442
Rosalind_2391 Rosalind_2323


## O(nlogn) 🚧

In [19]:
import bisect

def build_suffix_array(strings: list[str]) -> list[Tuple[str, int]]:
    """
    Build a suffix array for a list of strings. 
    Each entry in the suffix array is a tuple (suffix, index) 
    where 'index' is the index of the string from which the suffix originates.
    """
    suffixes = []
    for idx, s in enumerate(strings):
        for i in range(len(s)):
            suffixes.append((s[i:], idx))
    suffixes.sort()
    return suffixes

def overlap_graph_bisect(data: str, k: int = 3) -> list[tuple[str, str]]:
    """
    Find the overlap graph for the given DNA strings in FASTA format using a suffix array and bisect module.
    """
    parsed_data = parse_fasta(data)
    labels = [label for label, _ in parsed_data.items()]
    sequences = [seq for _, seq in parsed_data.items()]

    # Build the suffix array
    suffix_array = build_suffix_array(sequences)

    adjacency_list = []
    for i, seq in enumerate(sequences):
        # Extract the k-mer from the end of the sequence
        kmer = seq[-k:]
        
        # Use bisect_left to find the position of the k-mer in the suffix array
        pos = bisect.bisect_left(suffix_array, (kmer,))
        while pos < len(suffix_array) and suffix_array[pos][0][:k] == kmer:
            # Ensure the matching entry is a prefix of another sequence
            if suffix_array[pos][1] != i and suffix_array[pos][0].startswith(kmer) and len(suffix_array[pos][0]) > k:
                edge = (labels[i], labels[suffix_array[pos][1]])
                if edge not in adjacency_list:
                    adjacency_list.append(edge)
            pos += 1
                
    return adjacency_list

for ans in overlap_graph_bisect(fasta):
    print(*ans)


Rosalind_0498 Rosalind_0442
Rosalind_0498 Rosalind_2391
Rosalind_2391 Rosalind_2323
